In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [3]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')


In [7]:
fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
fake['class']=0
true['class']=1

In [9]:
data = pd.concat([fake,true], axis = 0)

In [10]:
data.sample(10)

,title,text,subject,date,class
3555,Trump officials under fire over vague tax prop...,WASHINGTON (Reuters) - Missing from President ...,politicsNews,"May 25, 2017",1
5155,Rio Olympics Opening Ceremonies Had A Big Mes...,Conservatives have something to say about abso...,News,"August 6, 2016",0
8426,All The ‘Proof’ Republicans Need To Confirm O...,"Since President Obama took office in 2009, the...",News,"January 30, 2016",0
3858,"Russia Drops Bombshell, Confirms It’s Been Ta...",For the entirety of Donald Trump s presidentia...,News,"November 10, 2016",0
7667,SNL Perfectly SLAMS Trump Supporters With ‘Ra...,If you haven t noticed by now that Donald Trum...,News,"March 6, 2016",0
14498,FEEL THE BERN….How Hillary Walked Away From NH...,"First she won the coin toss in Iowa, and now H...",politics,"Feb 10, 2016",0
10733,AUTHOR OF CHILDREN’S BOOKS Gets DESTROYED On S...,"Children s author and father of two, Ken Jenni...",politics,"May 31, 2017",0
953,New York Times Reporter Tells Of Horribly Deg...,Donald Trump has never tried to hide the fact ...,News,"July 1, 2017",0
10160,Sanders says party leaders may 'rethink' after...,WASHINGTON (Reuters) - Fresh from Democratic p...,politicsNews,"March 27, 2016",1
21219,CHURCH REPLACES JESUS In Nativity Scene With D...,Because the tragic drowning of this Muslim Syr...,left-news,"Dec 16, 2015",0


In [11]:
data = data.drop(["title", "subject","date"], axis = 1)


In [12]:
data.reset_index(inplace=True)

In [13]:
data.drop(['index'],axis = 1,inplace=True)

In [14]:
data.sample(5)

,text,class
23415,Jamie Ross Courthouse News ServicePHOENIX Tw...,0
31744,VIENTIANE (Reuters) - The Philippines said on ...,1
26305,WASHINGTON (Reuters) - U.S. President Donald T...,1
17521,"One month ago, CNN political analyst and Ameri...",0
11645,Here is actor Robert John Davi s brilliant let...,0


In [15]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]',"",text)
    text = re.sub("\\W","",text)
    text = re.sub("https?:://\S+|www\.\S+","",text)
    text = re.sub("<.*?>+","",text)
    text = re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text = re.sub("\n","",text)
    text = re.sub("\w*\d\w*","",text)
    return text


<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\shish\AppData\Local\Temp\ipykernel_15188\3148511301.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]',"",text)
C:\Users\shish\AppData\Local\Temp\ipykernel_15188\3148511301.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub("https?:://\S+|www\.\S+","",text)
C:\Users\shish\AppData\Local\Temp\ipykernel_15188\3148511301.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub("\w*\d\w*","",text)


In [16]:
data["text"] = data["text"].apply(clean_text)

In [17]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25,random_state=42)

In [18]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [19]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [20]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.5251670378619153

In [21]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69      5895
           1       0.00      0.00      0.00      5330

    accuracy                           0.53     11225
   macro avg       0.26      0.50      0.34     11225
weighted avg       0.28      0.53      0.36     11225



c:\fakenewsdetection\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\fakenewsdetection\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\fakenewsdetection\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']